# Random Forest Classifier with Pipeline And Hyperparameter Tuning

In [1]:
import seaborn as sns
df=sns.load_dataset('tips')

In [2]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [3]:
df['day'].unique()

['Sun', 'Sat', 'Thur', 'Fri']
Categories (4, object): ['Thur', 'Fri', 'Sat', 'Sun']

In [4]:
df['time'].unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [5]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [6]:
df.describe()

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


In [7]:
from sklearn.preprocessing import LabelEncoder

In [8]:
encoder=LabelEncoder()

In [9]:
df['time']=encoder.fit_transform(df['time'])

In [10]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,0,2
1,10.34,1.66,Male,No,Sun,0,3
2,21.01,3.50,Male,No,Sun,0,3
3,23.68,3.31,Male,No,Sun,0,2
4,24.59,3.61,Female,No,Sun,0,4


In [11]:
df['time'].unique()

array([0, 1])

In [12]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,0,2
1,10.34,1.66,Male,No,Sun,0,3
2,21.01,3.50,Male,No,Sun,0,3
3,23.68,3.31,Male,No,Sun,0,2
4,24.59,3.61,Female,No,Sun,0,4


In [13]:
X=df.drop(labels=['time'],axis=1)
y=df.time

In [14]:
X.head()

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4


In [15]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: time, dtype: int64

In [16]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=42)

In [17]:
X_train.shape

(195, 6)

In [18]:
X_test.shape

(49, 6)

In [19]:
y_train.shape

(195,)

In [20]:
y_test.shape

(49,)

# Feature Engineering Automation

In [21]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer #handling missing values
from sklearn.preprocessing import StandardScaler #Feature Scaling
from sklearn.preprocessing import OneHotEncoder #categorical to numerical
from sklearn.compose import ColumnTransformer

In [22]:
# Define column lists
numerical_cols = ['total_bill', 'tip', 'size']
categorical_cols = ['sex', 'smoker', 'day']


In [23]:
# Numerical pipeline
num_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

In [24]:
# Categorical pipeline
cat_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

In [25]:
# Combine both pipelines using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num_pipeline', num_pipeline, numerical_cols),
        ('cat_pipeline', cat_pipeline, categorical_cols)
    ]
)

In [26]:
preprocessor

ColumnTransformer(transformers=[('num_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['total_bill', 'tip', 'size']),
                                ('cat_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoder',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['sex', 'smoker', 'day'])])

In [27]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)


# Model building Using Random Forest

In [28]:
from sklearn.ensemble import RandomForestClassifier

In [29]:
rfc=RandomForestClassifier()

In [30]:
rfc.fit(X_train,y_train)

RandomForestClassifier()

In [31]:
y_pred=rfc.predict(X_test)

In [32]:
y_pred

array([0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 1])

In [33]:
y_test

24     0
6      0
153    0
211    0
198    1
176    0
192    1
124    1
9      0
101    0
45     0
233    0
117    1
177    0
82     1
146    1
200    1
15     0
66     0
142    1
33     0
19     0
109    0
30     0
186    0
120    1
10     0
73     0
159    0
156    0
112    0
218    0
25     0
60     0
18     0
119    1
97     0
197    1
139    1
241    0
75     0
127    1
113    0
16     0
196    1
67     0
168    0
38     0
195    1
Name: time, dtype: int64

In [34]:
from sklearn.metrics import classification_report,accuracy_score

In [35]:
print(accuracy_score(y_pred,y_test))

0.9591836734693877


# hyperparameter tunning

In [36]:
classifier=RandomForestClassifier()

In [37]:
params={
    'max_depth':[3,5,10,None],
    'n_estimators':[100,200,300],
    'criterion':['gini','entropy']
}

In [38]:
from sklearn.model_selection import RandomizedSearchCV

In [39]:
cv=RandomizedSearchCV(classifier,param_distributions=params,scoring='accuracy',cv=5,verbose=3)

In [40]:
cv.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=entropy, max_depth=3, n_estimators=100;, score=0.974 total time=   0.1s
[CV 2/5] END criterion=entropy, max_depth=3, n_estimators=100;, score=0.949 total time=   0.1s
[CV 3/5] END criterion=entropy, max_depth=3, n_estimators=100;, score=0.974 total time=   0.1s
[CV 4/5] END criterion=entropy, max_depth=3, n_estimators=100;, score=0.923 total time=   0.1s
[CV 5/5] END criterion=entropy, max_depth=3, n_estimators=100;, score=0.949 total time=   0.1s
[CV 1/5] END criterion=entropy, max_depth=None, n_estimators=300;, score=0.974 total time=   0.4s
[CV 2/5] END criterion=entropy, max_depth=None, n_estimators=300;, score=0.923 total time=   0.5s
[CV 3/5] END criterion=entropy, max_depth=None, n_estimators=300;, score=1.000 total time=   0.5s
[CV 4/5] END criterion=entropy, max_depth=None, n_estimators=300;, score=0.949 total time=   0.4s
[CV 5/5] END criterion=entropy, max_depth=None, n_estimators=300;, score

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10, None],
                                        'n_estimators': [100, 200, 300]},
                   scoring='accuracy', verbose=3)

In [41]:
cv.best_params_

{'n_estimators': 200, 'max_depth': 3, 'criterion': 'gini'}

# model building Automation

In [42]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [43]:
models={
    'Random Forest':RandomForestClassifier(),
    'Logistic Regression':LogisticRegression(),
    'Decision Tree':DecisionTreeClassifier()
}

In [44]:
from sklearn.metrics import accuracy_score

In [47]:
def evaluate_model(X_train,X_test,y_train,y_test,models):
    report={}
    for i in range(len(models)):
        model=list(models.values())[i]
        #train model
        model.fit(X_train,y_train)
        
        #Predict testing model
        y_test_pred=model.predict(X_test)
        test_model_score=accuracy_score(y_test,y_test_pred)
        report[list(models.keys())[i]]=test_model_score
    return report    





In [49]:
evaluate_model(X_train,X_test,y_train,y_test,models)

{'Random Forest': 0.9591836734693877,
 'Logistic Regression': 1.0,
 'Decision Tree': 0.9387755102040817}